In [2]:
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

from azureml.core import Environment, Model, Workspace
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

load_dotenv()
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
AZURE_RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP")
AZURE_WORKSPACE_NAME = os.getenv("AZURE_WORKSPACE_NAME")


version = str(uuid.uuid4())[:4]

source_directory = "./azureml"
experiment_name = "oc-p8-experiment-1"


model_choices = [
    "unet_xception",
    "deeplab_v3plus",
    "fcn_8",
]
augment_choices = [
    False,
    True,
]
resize_choices = [
    64,
    # 80,
    128,
    # 160,
    256,
    # 320,
    512,
    # 640,
]


# connect to your workspace
ws = Workspace(
    subscription_id=AZURE_SUBSCRIPTION_ID,
    resource_group=AZURE_RESOURCE_GROUP,
    workspace_name=AZURE_WORKSPACE_NAME,
)


env_name = experiment_name + "-predict"
try:
    env = Environment.get(workspace=ws, name=env_name)
except:
    env = Environment.from_conda_specification(
        name=env_name,
        file_path=Path(source_directory, "conda_dependencies.yml"),
    )
    env.inferencing_stack_version = "latest"
    env.register(workspace=ws)


inference_config = InferenceConfig(
    source_directory=source_directory,
    entry_script="predict.py",
    environment=env,
)


for resize in resize_choices:
    for model in model_choices:
        for augment in augment_choices:
            model_name = f"{model}_{resize}{'_augment' if augment else ''}"
            aci_config = AciWebservice.deploy_configuration(
                cpu_cores=3.8,
                memory_gb=15,
                auth_enabled=True,
            )

            aml_model = Model(ws, model_name)

            service = Model.deploy(
                workspace=ws,
                name=model_name.replace("_", "-") + "-" + version,
                models=[aml_model],
                inference_config=inference_config,
                deployment_config=aci_config,
                overwrite=True,
                show_output=True,
            )

            # service.wait_for_deployment(show_output=True)


Models: ['unet_xception_64:7']
Entry script: azureml/predict.py
Environment variables: {'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE', 'AZUREML_SOURCE_DIRECTORY': 'azureml', 'AZUREML_ENTRY_SCRIPT': 'azureml/predict.py'}
Environment dependencies: ['python=3.8', {'pip': ['tensorflow==2.7.0', 'azureml-defaults', 'azureml-sdk', 'inference-schema', 'numpy', 'matplotlib', 'Pillow']}]
Environment docker image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220208.v1
CPU requirement: 3.8, Memory requirement: 15GB
Uploading dependency /tmp/tmpeodg3_id/6cb01e91.tar.gz.
Request submitted, please run wait_for_deployment(show_output=True) to get deployment status.
Models: ['unet_xception_64_augment:9']
Entry script: azureml/predict.py
Environment variables: {'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE', 'AZUREML_SOURCE_DIRECTORY': 'azureml', 'AZUREML_ENTRY_SCRIPT': 'azureml/predict.py'}
Environment dependencies: ['python=3.8', {'pip': ['tensorflow==2.7.0', 'azureml-defaults', 'azureml-sdk', 'inference-schema', 'n